In [1]:
import pennylane as qml
import numpy as np
import tensorflow as tf

In [2]:
dev = qml.device('cirq.simulator', wires=4)

In [3]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)

In [4]:
def generator(w, **kwargs):
    qml.Hadamard(wires=1)
    qml.RX(w[0], wires=1)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=1)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=1)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[1, 2])
    qml.RX(w[6], wires=1)
    qml.RY(w[7], wires=1)
    qml.RZ(w[8], wires=1)

In [7]:
def swap_test():
    qml.Hadamard(wires=3)
    qml.CSWAP(wires=[3, 0, 1])
    qml.Hadamard(wires=3)

In [11]:
@qml.qnode(dev, interface="tf")
def overlap(phi, theta, omega, gen_weights):
    real([phi, theta, omega])
    generator(gen_weights)
    swap_test()
    return qml.expval(qml.PauliZ(3))

In [9]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))

gen_weights = tf.Variable(init_gen_weights)

In [10]:
opt = tf.keras.optimizers.SGD(0.4)

In [12]:
cost = lambda: 1 - overlap(phi, theta, omega, gen_weights)
for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = 0.4184114336967468
Step 5: cost = 0.11036825180053711
Step 10: cost = 0.0188065767288208
Step 15: cost = 0.003579854965209961
Step 20: cost = 0.0007672309875488281
Step 25: cost = 0.00017178058624267578
Step 30: cost = 3.910064697265625e-05
Step 35: cost = 9.059906005859375e-06
Step 40: cost = 2.6226043701171875e-06
Step 45: cost = 7.748603820800781e-07


In [14]:
obs_real = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]
obs_gen = [qml.PauliX(1), qml.PauliY(1), qml.PauliZ(1)]

bloch_vector_real = qml.map(real, obs_real, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs_gen, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.21694186  0.45048442 -0.86602521]
Generator Bloch vector: [-0.21651354  0.45036009 -0.86619705]
